# Suspicious MsiExec Directory
Detects suspicious msiexec process starts in an uncommon directory

## Rule Content
```
- title: Suspicious MsiExec Directory
  id: e22a6eb2-f8a5-44b5-8b44-a2dbd47b1144
  status: experimental
  description: Detects suspicious msiexec process starts in an uncommon directory
  references:
  - https://twitter.com/200_okay_/status/1194765831911215104
  tags:
  - attack.defense_evasion
  - attack.t1036
  author: Florian Roth
  date: 2019/11/14
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image: '*\msiexec.exe'
    filter:
      Image:
      - C:\Windows\System32\\*
      - C:\Windows\SysWOW64\\*
      - C:\Windows\WinSxS\\*
    condition: selection and not filter
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:*\\msiexec.exe AND (NOT (process_path.keyword:(C\:\\Windows\\System32\\* OR C\:\\Windows\\SysWOW64\\* OR C\:\\Windows\\WinSxS\\*))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()